In [1]:
import numpy as np
import cv2

def gammaCorrection(src, gamma):
    invGamma = 1 / gamma
    table = [((i / 255) ** invGamma) * 255 for i in range(256)]
    table = np.array(table, np.uint8)
    return cv2.LUT(src, table)

def display_tracking_window_axis_center(frame,x,y,tw_width,tw_height, rows):
        bounding_box = cv2.rectangle(frame,
                  (x, y),
                  (x + tw_width, y + tw_height),
                  (255, 0, 0), 1)
        horisontal_axis = cv2.line(frame,
                        (x + int(tw_width/2), 0),
                        (x + int(tw_width/2), rows),
                        (255, 0, 0), 1)
        object_center = cv2.circle(frame,
                                (x + int(tw_width/2), y + int(tw_height/2)),
                                2, (0,0,255),2)
        
        return (bounding_box, horisontal_axis,object_center)

In [2]:
video = cv2.VideoCapture(r'.\input_data\2023_03_14_PTM.avi')

roi_start_x = 315
roi_end_x = 370
roi_start_y = 217
roi_end_y = 400

start_point_x = 30
start_point_y = 0
start_width = 25
start_height = 60
term_criteria= (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10,1)

while True:
    orig_frame = video.read()[1]
    org_frame = orig_frame[roi_start_x:roi_end_x, roi_start_y:roi_end_y]  
    frame = gammaCorrection(org_frame, 2)
    gray_frame = cv2.GaussianBlur(cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY),
                                  (5,5),0)
    threshold = cv2.threshold(gray_frame, 200,255, cv2.THRESH_BINARY)[1]
    track_window = cv2.meanShift(threshold,
                                 (start_point_x, start_point_y, start_width, start_height),
                                 term_criteria)[1]
    start_point_x, start_point_y, tw_width, tw_height = track_window
    rows, cols, _bands = frame.shape

    bounding_box, horisontal_axis, object_center = display_tracking_window_axis_center(
        frame,start_point_x, start_point_y,tw_width,tw_height,rows)

    cv2.imshow('Frame', frame)
    key = cv2.waitKey(5) #for framerate 20, normal playback is 100
    if key == 27:
        break
video.release()
cv2.destroyAllWindows()